### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


In [45]:
# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [46]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]  # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))  # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent  
    movie_mat = np.random.rand(latent_features, n_movies)  # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    squared_errors = []
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum += old_sse
        
        # For each user-movie pair
        for user in range(n_users):
            for movie in range(n_movies):
                rating = ratings_mat[user, movie]
                
                # if the rating exists
                if rating > 0:
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error = rating - np.dot(user_mat[user, :], movie_mat[:, movie])

                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum = np.sum(error**2)
                    squared_errors.append(sse_accum)
                    
                    for latent_feature in range(latent_features):
                        # update the values in each matrix in the direction of the gradient
                        user_mat[user, latent_feature] += learning_rate * error * movie_mat[latent_feature, movie]
                        movie_mat[latent_feature, movie] += learning_rate * error * user_mat[user, latent_feature]
                    
        mse = np.mean(squared_errors)

        # print results for iteration
        print("Iteration : {}, Mean Squared Error: {}".format(iteration, mse))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [47]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0005, iters=10) # use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
Iteration : 0, Mean Squared Error: 60.01788336208396
Iteration : 1, Mean Squared Error: 59.34666666707712
Iteration : 2, Mean Squared Error: 58.666383217338804
Iteration : 3, Mean Squared Error: 57.9766700585633
Iteration : 4, Mean Squared Error: 57.27726549922241
Iteration : 5, Mean Squared Error: 56.568011451144386
Iteration : 6, Mean Squared Error: 55.84885562733781
Iteration : 7, Mean Squared Error: 55.11985342969226
Iteration : 8, Mean Squared Error: 54.381169357122175
Iteration : 9, Mean Squared Error: 53.63307776581463


In [48]:
#Compare the predicted and actual results
predictions = np.dot(user_mat, movie_mat)
print(predictions)
print(ratings_mat)

[[ 1.95476179  2.40650033  1.49664974  1.28889236]
 [ 2.27478755  2.30453841  1.79849457  1.48733897]
 [ 2.42640425  2.87582847  1.87473951  1.63555803]
 [ 1.65845239  2.12239052  1.43080534  1.21480865]
 [ 1.7640318   2.47362451  1.27333945  1.05968129]
 [ 1.68302805  1.70625529  1.17853848  0.94292453]
 [ 2.24979637  2.67754062  1.76968713  1.41279505]
 [ 2.21935705  2.52811417  1.83256359  1.39820617]
 [ 2.6296861   3.0092956   2.12219321  1.76919678]
 [ 2.80280013  2.95838751  2.2014491   1.77324036]
 [ 2.07497849  2.64178687  1.77239375  1.52774324]
 [ 1.58540566  1.88836177  1.26730573  1.04879787]
 [ 2.06117198  2.90172719  1.60128856  1.40656718]
 [ 2.33310566  2.57475727  1.68928866  1.48280578]
 [ 2.31874871  2.98389511  2.05604516  1.65747884]
 [ 1.96430759  2.36019516  1.56618509  1.17124486]
 [ 1.96488332  2.66690034  1.6061896   1.3713343 ]
 [ 1.93408352  2.10543512  1.41203528  1.11955346]
 [ 1.62879683  1.91437726  0.98372584  0.80707784]
 [ 2.12911624  2.60589623  1.98

**Write your summary here.**

As we can see, our model perform poorly right now. The predictions are way off the mark when compared to the actual ratings. We will have to train our model longer and adjust some parameters.

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [49]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0005, iters=250)  #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
Iteration : 0, Mean Squared Error: 60.24829497352687
Iteration : 1, Mean Squared Error: 59.634270310930766
Iteration : 2, Mean Squared Error: 59.0092334189011
Iteration : 3, Mean Squared Error: 58.37285645124475
Iteration : 4, Mean Squared Error: 57.7248969574207
Iteration : 5, Mean Squared Error: 57.06520162316149
Iteration : 6, Mean Squared Error: 56.39370991342881
Iteration : 7, Mean Squared Error: 55.71045747235378
Iteration : 8, Mean Squared Error: 55.015579126924855
Iteration : 9, Mean Squared Error: 54.30931133586789
Iteration : 10, Mean Squared Error: 53.59199392333473
Iteration : 11, Mean Squared Error: 52.86407093957188
Iteration : 12, Mean Squared Error: 52.12609049849258
Iteration : 13, Mean Squared Error: 51.378703455689305
Iteration : 14, Mean Squared Error: 50.62266081033709
Iteration : 15, Mean Squared Error: 49.85880974079769
Iteration : 16, Mean Squared Error: 49.0880882163238
Iteration : 17, Mean Squared Error:

In [50]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.28446596   9.03322547  10.31271473  10.21527392]
 [  8.85763712   6.94427148   8.98238356   8.63940386]
 [  8.04637108   7.60348903   8.26463929   7.98604555]
 [  9.48597382   7.87261413  10.22083126   9.39739985]
 [  8.83104834   7.09324242   8.9493285    8.42259522]
 [  6.66028362   5.72606322   7.42789519   6.60829351]
 [  9.36801104   7.67977153   9.75674176   9.17211336]
 [  8.14276486   7.17380396   8.72925406   8.27141545]
 [  8.1170569    7.93202907   8.78156524   8.25085102]
 [  8.1482821    6.02851487   8.39868074   7.60572281]
 [  8.64060355   7.81761031   9.87093316   8.64658619]
 [  9.92024816   8.15444335  10.15171307   9.50351155]
 [  9.12134162   8.21953157  10.42083605   9.08734199]
 [  6.09697285   6.34085521   6.70242289   6.52754776]
 [  9.83649959   8.09361294  10.47000117   9.5846952 ]
 [  9.70970797   8.39854569  10.04276506   9.76580224]
 [  8.88237375   6.90541024   8.67126584   8.33802303]
 [  7.54591789   6.63438208   6.74127221   7.50033661]
 [  8.0065

**Write your summary here.**
As we can see, by training our model longer, our model perform much better. The MSE dropped down to approximately 7, compared to its previous value of 53.

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [51]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [52]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0005, iters=250)  #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
Iteration : 0, Mean Squared Error: 55.4805386049788
Iteration : 1, Mean Squared Error: 54.80705346497345
Iteration : 2, Mean Squared Error: 54.122859682057175
Iteration : 3, Mean Squared Error: 53.42803011763181
Iteration : 4, Mean Squared Error: 52.72273303060837
Iteration : 5, Mean Squared Error: 52.007233739445915
Iteration : 6, Mean Squared Error: 51.281895619461885
Iteration : 7, Mean Squared Error: 50.547180297452165
Iteration : 8, Mean Squared Error: 49.80364691394672
Iteration : 9, Mean Squared Error: 49.05195033664936
Iteration : 10, Mean Squared Error: 48.29283822709822
Iteration : 11, Mean Squared Error: 47.52714688641828
Iteration : 12, Mean Squared Error: 46.7557958350033
Iteration : 13, Mean Squared Error: 45.97978111451425
Iteration : 14, Mean Squared Error: 45.20016733781183
Iteration : 15, Mean Squared Error: 44.41807855211146
Iteration : 16, Mean Squared Error: 43.63468802120859
Iteration : 17, Mean Squared Erro

In [53]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.374813416514108:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [54]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)  #fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
Iteration : 0, Mean Squared Error: 30.817772102547575
Iteration : 1, Mean Squared Error: 23.62231589431251
Iteration : 2, Mean Squared Error: 19.635349141096402
Iteration : 3, Mean Squared Error: 17.03461043326778
Iteration : 4, Mean Squared Error: 15.171120674764998
Iteration : 5, Mean Squared Error: 13.752024308860541
Iteration : 6, Mean Squared Error: 12.62460117302913
Iteration : 7, Mean Squared Error: 11.700724566820043
Iteration : 8, Mean Squared Error: 10.925601744584556
Iteration : 9, Mean Squared Error: 10.263160099812493
Iteration : 10, Mean Squared Error: 9.688553116508588
Iteration : 11, Mean Squared Error: 9.184029252477538
Iteration : 12, Mean Squared Error: 8.736517379859132
Iteration : 13, Mean Squared Error: 8.336145764873956
Iteration : 14, Mean Squared Error: 7.975296516919993
Iteration : 15, Mean Squared Error: 7.647981189417341
Iteration : 16, Mean Squared Error: 7.349416281393605
Iteration : 17, Mean Squared

`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [59]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users)) # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

ratings_for_missing = np.count_nonzero(np.isnan(first_1000_users))  # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [60]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
